# Assignment 5, Question 6: Data Transformation

**Points: 20**

Transform and engineer features from the clinical trial dataset.

## Setup

In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import utilities
from q3_data_utils import load_data, clean_data, transform_types, create_bins, fill_missing

df = load_data('data/clinical_trial_raw.csv')
#df = load_data('output/q5_cleaned_data.csv')
print(f"Loaded {len(df)} patients")

# Prewritten visualization functions for transformation analysis
def plot_distribution(series, title, figsize=(10, 6)):
    """
    Create a histogram of a numeric series.
    
    Args:
        series: pandas Series with numeric data
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.hist(bins=30)
    plt.title(title)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

def plot_value_counts(series, title, figsize=(10, 6)):
    """
    Create a bar chart of value counts.
    
    Args:
        series: pandas Series with value counts
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.plot(kind='bar')
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


Loaded 10000 patients


## Part 1: Type Conversions (5 points)

1. Convert 'enrollment_date' to datetime using the `transform_types()` utility
2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
3. Ensure all numeric columns are proper numeric types
4. Display the updated dtypes

In [134]:
# TODO: Type conversions
# 1. Use transform_types() to convert enrollment_date to datetime
# 2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
# 3. Ensure all numeric columns are proper numeric types
# 4. Display the updated dtypes using df.dtypes
type_map = {'enrollment_date': 'datetime'}
df = transform_types(df, type_map)
type_map = {'site': 'category', 'intervention_group': 'category', 'sex': 'category'}
df = transform_types(df, type_map)

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.dtypes

patient_id                    object
age                            int64
sex                         category
bmi                          float64
enrollment_date       datetime64[ns]
systolic_bp                  float64
diastolic_bp                 float64
cholesterol_total            float64
cholesterol_hdl              float64
cholesterol_ldl              float64
glucose_fasting              float64
site                        category
intervention_group          category
follow_up_months               int64
adverse_events                 int64
outcome_cvd                   object
adherence_pct                float64
dropout                       object
dtype: object

## Part 2: Feature Engineering (8 points)

Create these new calculated columns:

1. `cholesterol_ratio` = cholesterol_ldl / cholesterol_hdl
2. `bp_category` = categorize systolic BP:
   - 'Normal': < 120
   - 'Elevated': 120-129
   - 'High': >= 130
3. `age_group` using `create_bins()` utility:
   - Bins: [0, 40, 55, 70, 100]
   - Labels: ['<40', '40-54', '55-69', '70+']
4. `bmi_category` using standard BMI categories:
   - Underweight: <18.5
   - Normal: 18.5-24.9
   - Overweight: 25-29.9
   - Obese: >=30

In [135]:
# TODO: Calculate cholesterol ratio
df['cholesterol_ratio'] = df['cholesterol_total'] / df['cholesterol_hdl']

In [136]:
# TODO: Categorize blood pressure
df['bp_category'] = pd.cut(df['systolic_bp'], 
                           bins=[0, 120, 129, float('inf')], 
                           labels=['Normal', 'Elevated', 'High'])

**Note:** The `create_bins()` function has an optional `new_column` parameter. If you don't specify it, the new column will be named `{original_column}_binned`. You can use `new_column='age_group'` to give it a custom name.


In [137]:
# TODO: Create age groups
df = create_bins(df,
                 column='age',
                 bins=[0, 40, 55, 70, 100],
                 labels=['<40', '40-54', '55-69', '70+'],
                 new_column='age_group')

In [138]:
# TODO: Create BMI categories
df['bmi_category'] = pd.cut(df['bmi'],
                            bins=[0, 18.5, 24.9, 29.9, float('inf')], 
                            labels=['Underweight', 'Normal', 'Overweight', 'Obese'])

## Part 3: String Cleaning (2 points)

If there are any string columns that need cleaning:
1. Convert to lowercase
2. Strip whitespace
3. Replace any placeholder values

In [139]:
# TODO: String cleaning
df['outcome_cvd'] = df['outcome_cvd'].str.strip().str.lower()
df['outcome_cvd'].unique()

df['site'] = df['site'].str.lower().str.strip()
df['site'] = df['site'].replace({'site  a': 'site a', 'site_d': 'site d'})

df['intervention_group'] = df['intervention_group'].str.lower().str.strip()
df['intervention_group'] = df['intervention_group'].replace({'contrl': 'control'})
df['intervention_group'] = df['intervention_group'].replace(['treatmen a', 'treatmenta'], 'treatment a')
df['intervention_group'] = df['intervention_group'].replace({'treatment  b': 'treatment b'})

df['sex'] = df['sex'].str.lower().str.strip()
df['sex'] = df['sex'].replace({'m':'male', 'f':'female'})

## Part 4: One-Hot Encoding (5 points)

Create dummy variables for categorical columns:
1. One-hot encode 'intervention_group' using `pd.get_dummies()`
2. One-hot encode 'site'
3. Drop the original categorical columns
4. Show the new shape and column names

In [140]:
# TODO: One-hot encoding
group_dummies = pd.get_dummies(df['intervention_group'], prefix='intervention_group', dtype='int64')
site_dummies = pd.get_dummies(df['site'], prefix='site', dtype='int64')
df_transformed = pd.concat([df, group_dummies, site_dummies], axis=1)
df_transformed.drop(['intervention_group', 'site'], axis=1, inplace=True)
print(df_transformed.columns, df_transformed.shape)

Index(['patient_id', 'age', 'sex', 'bmi', 'enrollment_date', 'systolic_bp',
       'diastolic_bp', 'cholesterol_total', 'cholesterol_hdl',
       'cholesterol_ldl', 'glucose_fasting', 'follow_up_months',
       'adverse_events', 'outcome_cvd', 'adherence_pct', 'dropout',
       'cholesterol_ratio', 'bp_category', 'age_group', 'bmi_category',
       'intervention_group_control', 'intervention_group_treatment a',
       'intervention_group_treatment b', 'site_site a', 'site_site b',
       'site_site c', 'site_site d', 'site_site e'],
      dtype='object') (10000, 28)


## Part 5: Save Transformed Data

Save the fully transformed dataset to `output/q6_transformed_data.csv`

In [141]:
# TODO: Save transformed data
# df_transformed.to_csv('output/q6_transformed_data.csv', index=False)
df_transformed.to_csv('output/q6_transformed_data.csv', index=False)